## 19. Packages, `__init__.py`, and namespace packages

A *package* is just a folder placed on `sys.path` and containing an `__init__.py` file. Importing `import pack.mod` executes `pack/__init__.py` first, then `pack/mod.py`.  A missing `__init__.py` turns the folder into a *namespace package*—handy for splitting large libs across repos but still letting you `import mypkg.sub` anywhere. Common gotcha: two folders of the same package name earlier on `sys.path` may shadow each other.

In [ ]:
# Simulate a tiny package at runtime for demo purposes
import types, sys
mypkg = types.ModuleType('mypkg')
mypkg.__path__ = []  # mark as package
def _hello():
    return 'hi from mypkg'
mypkg.hello = _hello
sys.modules['mypkg'] = mypkg

import mypkg
print(mypkg.hello())

### Quick check

1. T / F `import pack` executes files inside every directory named `pack` on `sys.path`.

2. What does **absence** of `__init__.py` create?
  a. regular package  b. namespace package

<details><summary>Answer key</summary>

1. **False** — only the first matching directory.
2. **b** — folder without `__init__.py` becomes namespace package.

</details>

## 20. Error handling with `try` / `except`

Exceptions bubble upward until caught. A narrow `except ValueError` beats the anti‑pattern `except:` which swallows `KeyboardInterrupt` and debugging clues. Use `as err` to inspect the object, log it, or re‑raise.

In [ ]:
try:
    int('not‑an‑int')
except ValueError as e:
    print('Caught:', e)

### Quick check

1. Which except block is safer?
  a. `except:`  b. `except Exception:`

2. T / F After handling, execution continues **after** the `except` suite.

<details><summary>Answer key</summary>

1. **b** — at least it lets `SystemExit/KeyboardInterrupt` through.
2. **True** — unless you `raise` again.

</details>

## 21. `else`, `finally`, and raising custom exceptions

`else` runs only when no exception fired; useful for code that *depends* on success. `finally` always runs—close files, release locks here. Create domain‑specific errors by subclassing `Exception`.

In [ ]:
class NegativeError(ValueError):
    pass

def sqrt(x):
    if x < 0:
        raise NegativeError('no negatives')
    return x ** 0.5

try:
    result = sqrt(4)
except NegativeError:
    result = None
else:
    print('Success', result)
finally:
    print('always runs')

### Quick check

1. `else` executes when:
  a. exception handled  b. no exception occurred

2. T / F `finally` is skipped when process receives `KeyboardInterrupt`.

<details><summary>Answer key</summary>

1. **b** — only on clean run.
2. **False** — it runs before unwinding (unless os.kill exit).

</details>

## 22. Built‑in helpers: `enumerate`, `zip`, `any`, `all`, `sorted`

These tiny functions remove boilerplate loops. `enumerate(lst)` yields `(index,val)` pairs; `zip(a,b)` stops at shortest input; `any()` and `all()` short‑circuit. Use `sorted(iterable, key=func)` instead of `list.sort()` when you need a new list.

In [ ]:
letters = ['a','b','c']
for i, ch in enumerate(letters):
    print(i, ch)

pairs = list(zip([1,2], ['one','two','three']))
print(pairs)
print(any(n>3 for n in [1,4,2]))

### Quick check

1. T / F `zip` pads shorter sequences with `None`.

2. `any([])` returns:
  a. True  b. False

<details><summary>Answer key</summary>

1. **False** — it truncates.
2. **b** — an empty iterable is *Falsey* so `any` returns False.

</details>

## 23. Iterator protocol: `iter()` / `next()`

When Python needs to loop it calls `iter(obj)`; that returns an iterator object with `__next__()`. Calling `next(it)` fetches the next item or raises `StopIteration`. Implementing these two dunders lets *any* object become iterable.

In [ ]:
class Countdown:
    def __init__(self, start):
        self.cur = start
    def __iter__(self):
        return self
    def __next__(self):
        if self.cur <= 0:
            raise StopIteration
        self.cur -= 1
        return self.cur+1

for n in Countdown(3):
    print(n)

### Quick check

1. What triggers `StopIteration` in a for‑loop?
  a. exhausted iterator  b. break statement

2. T / F `range(3)` returns an iterator directly.

<details><summary>Answer key</summary>

1. **a** — `for` catches it to end loop.
2. **False** — `range` is iterable but not itself an iterator.

</details>

## 24. Generator functions and `yield`

When a function contains `yield`, Python builds a *generator object* instead of running immediately. Each call to `next()` resumes where it left off—perfect for streams or infinite sequences.

In [ ]:
def fib():
    a,b = 0,1
    while True:
        yield a
        a,b = b,a+b

f = fib()
print([next(f) for _ in range(6)])

### Quick check

1. T / F `return` inside a generator stops iteration with `StopIteration`.

2. `yield` inside a loop preserves state between calls?
  a. Yes  b. No

<details><summary>Answer key</summary>

1. **True**.
2. **a** — local variables persist.

</details>

## 25. Generator expressions & memory efficiency

Replace `[x*x for x in big]` with `(x*x for x in big)` to stream values one‑by‑one, saving RAM. Common gotcha: once consumed, the generator is exhausted; you can’t iterate again unless you recreate it.

In [ ]:
gen = (n*n for n in range(5))
print(sum(gen))
print(list(gen))  # now empty

### Quick check

1. T / F `list(gen)` after `sum(gen)` still has elements.

2. Which comprehension holds **all** items in memory?
  a. `[x for x in it]`  b. `(x for x in it)`

<details><summary>Answer key</summary>

1. **False** — generator already exhausted.
2. **a** — list‑comprehension builds list.

</details>

## 26. `yield from` and generator delegation

`yield from subgen` forwards values, `.send()`, and exceptions to another generator without boilerplate loops. Great for coroutine pipelines or flattening nested generators.

In [ ]:
def sub():
    yield from (1,2,3)
    yield 'done'

print(list(sub()))

### Quick check

1. T / F `yield from` handles `StopIteration` automatically.

2. Delegation is useful to:
  a. duplicate items  b. compose generators

<details><summary>Answer key</summary>

1. **True**.
2. **b** — compose/flatten.

</details>

## 27. Defining classes & creating instances

Classes bundle data **and** behaviour. `__init__` initialises each instance; omit a return. Use `object` as implicit base in Python 3. A common newbie error is forgetting `self` as first parameter.

In [ ]:
class Dog:
    def __init__(self, name):
        self.name = name
    def bark(self):
        return f'{self.name} says woof!'

print(Dog('Fido').bark())

### Quick check

1. T / F `__init__` may return a value.

2. Which call constructs an instance?
  a. `Dog`  b. `Dog('Rex')`

<details><summary>Answer key</summary>

1. **False** — returning anything raises `TypeError`.
2. **b** — call the class.

</details>

## 28. Instance vs. class variables

Variables assigned at class body level are *shared* across all instances; those set as `self.attr` inside `__init__` belong to each object. Accidentally mutating a class‑level list changes it for everyone.

In [ ]:
class Counter:
    total = 0  # class var
    def __init__(self):
        Counter.total += 1

c1, c2 = Counter(), Counter()
print(Counter.total)

### Quick check

1. T / F Modifying `Counter.total` inside an instance affects all instances.

2. An attribute created **only** with `self.x = 1` lives:
  a. on instance  b. on class

<details><summary>Answer key</summary>

1. **True**.
2. **a** — instance dict.

</details>

## 29. Methods and the `self` parameter

`self` is just a convention for the *instance being operated on*. Python passes it automatically; forgetting it leads to *TypeError: missing positional ‘self’*.

In [ ]:
class Box:
    def volume(self, w, h, d):
        return w*h*d

b = Box()
print(b.volume(2,3,4))

### Quick check

1. T / F `self` is a reserved keyword.

2. Which call fails?
  a. `Box.volume(b,1,2,3)`  b. `b.volume(1,2,3)`  c. `Box.volume(1,2,3)`

<details><summary>Answer key</summary>

1. **False** — just a naming convention.
2. **c** — missing explicit instance.

</details>

## 30. Single inheritance & `super()`

Inheritance re‑uses behaviour. `super().__init__()` calls the parent initialiser; forgetting it can leave base state uninitialised. In simple single‑parent cases `super()` with no args is fine (Python 3).

In [ ]:
class Animal:
    def __init__(self, name):
        self.name = name
class Cat(Animal):
    def __init__(self, name, lives=9):
        super().__init__(name)
        self.lives = lives

print(Cat('Whiskers').name)

### Quick check

1. T / F `super()` resolves parent using the MRO.

2. `super()` inside a subclass without parent `__init__`: 
  a. is optional  b. raises AttributeError

<details><summary>Answer key</summary>

1. **True**.
2. **a** — if base has no `__init__`, call unnecessary.

</details>

## 31. String representation dunders

Implement `__str__` for *user‑friendly* output and `__repr__` for *developer* debugging. Guideline: `repr(obj)` should, when feasible, be valid Python code to recreate the object.

In [ ]:
class Point:
    def __init__(self,x,y):
        self.x,self.y=x,y
    def __str__(self):
        return f'({self.x},{self.y})'
    __repr__ = __str__

p=Point(2,3)
print(p)   # -> (2,3)
print([p]) # uses repr

### Quick check

1. `repr(obj)` is aimed at:
  a. end‑user  b. developer

2. T / F If `__repr__` missing, Python falls back to `__str__`.

<details><summary>Answer key</summary>

1. **b**.
2. **False** — opposite: `__str__` falls back to `__repr__`.

</details>

## 32. Emulating collections (`__len__`, `__getitem__`)

Implementing these dunders lets custom classes behave like lists/dicts. Useful in data‑science wrappers, config objects, etc. Forgetting `__iter__` forces Python to fallback to repeatedly indexing 0…N until `IndexError`.

In [ ]:
class Range2:
    def __init__(self, stop):
        self.stop=stop
    def __len__(self):
        return self.stop
    def __getitem__(self, index):
        if 0<=index<self.stop:
            return index
        raise IndexError

r = Range2(5)
print(len(r), r[3])
print(list(r))

### Quick check

1. T / F `for x in r` works without `__iter__` as long as `__len__` and `__getitem__` exist.

2. `__len__` must return:
  a. int  b. any numeric

<details><summary>Answer key</summary>

1. **True** — Python uses 0… until IndexError fallback.
2. **a** — must be `int`.

</details>

## 33. Operator overloading (`__add__`, `__eq__`, …)

Custom classes can define arithmetic, comparison, or container operations. Only overload when it makes semantic sense; mismatched behaviour surprises users.

In [ ]:
class Vector:
    def __init__(self,x,y):
        self.x,self.y=x,y
    def __add__(self,other):
        return Vector(self.x+other.x, self.y+other.y)
    def __repr__(self):
        return f'Vector({self.x},{self.y})'

print(Vector(1,2)+Vector(3,4))

### Quick check

1. `__eq__` should return:
  a. bool  b. any value

2. T / F If `__lt__` defined, `sort()` automatically understands `>` comparisons.

<details><summary>Answer key</summary>

1. **a**.
2. **False** — implement full set or use `functools.total_ordering`.

</details>

## 34. Properties & `@property`

`@property` turns attribute access into method calls: compute‑on‑access, validation, lazy loading. Overuse harms performance; use only where invariants matter.

In [ ]:
class Celsius:
    def __init__(self,temp):
        self._temp=temp
    @property
    def temp(self):
        return self._temp
    @temp.setter
    def temp(self,value):
        if value < -273.15:
            raise ValueError('below absolute zero')
        self._temp=value

c=Celsius(0)
c.temp = 25
print(c.temp)

### Quick check

1. T / F `@property` can define read‑only attributes.

2. Setter runs when:
  a. reading attr  b. assigning attr

<details><summary>Answer key</summary>

1. **True** — omit setter.
2. **b**.

</details>

## 35. Class methods vs. static methods

`@classmethod` receives the *class* as first arg (`cls`)—great for alternative constructors. `@staticmethod` gets no implicit first arg; use for utility funcs logically grouped with the class.

In [ ]:
class Pizza:
    def __init__(self, size):
        self.size=size
    @classmethod
    def large(cls):
        return cls(16)
    @staticmethod
    def inches_to_cm(inch):
        return inch*2.54

print(Pizza.large().size)
print(Pizza.inches_to_cm(12))

### Quick check

1. T / F `@staticmethod` can access class via `cls`.

2. `@classmethod` is invoked on instance?
  a. yes  b. also yes (works on both)

<details><summary>Answer key</summary>

1. **False** — no implicit arg.
2. **b** — callable on class *or* instance.

</details>

## 36. Multiple inheritance & the MRO

When subclassing multiple parents, Python uses the *Method Resolution Order* (C3 linearisation) to decide which method wins. Diagnose with `Class.mro()`. Diamond patterns require cooperative `super()` calls on all classes.

In [ ]:
class A: pass
class B(A): pass
class C(A): pass
class D(B,C): pass
print([cls.__name__ for cls in D.mro()])

### Quick check

1. T / F `super()` always calls *first* parent in the tuple.

2. MRO ensures:
  a. deterministic lookup  b. random lookup

<details><summary>Answer key</summary>

1. **False** — depends on C3 order.
2. **a** — deterministic.

</details>

## 37. Composition vs. inheritance (design)

Prefer *has‑a* (composition) over *is‑a* (inheritance) when behaviour is reused but identity differs. Over‑deep inheritance trees lead to fragile coupling. Strategy: compose small classes and expose required subset.

In [ ]:
class Engine:
    def start(self):print('vroom')
class Car:
    def __init__(self):
        self.engine=Engine()
    def drive(self):
        self.engine.start()
        print('go')

Car().drive()

### Quick check

1. T / F Composition lets you swap components at runtime.

2. Deep inheritance can cause:
  a. tight coupling  b. loose coupling

<details><summary>Answer key</summary>

1. **True**.
2. **a**.

</details>